In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity, cosine_distances
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import squarify
import seaborn as sns
from surprise import accuracy
from surprise.model_selection.validation import cross_validate
from surprise.dataset import Dataset
from surprise.reader import Reader
from surprise import SVD, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split
import random

## Calling the Saved Dataset

In [2]:
course_dataset = pd.read_csv("data/courseData.csv", encoding= 'unicode_escape')
course_dataset.head()

Unnamed: 0   sr_ degree_1 degree_1_specializations  campus key_skills  \
0           0  1001     B.E.               Mechanical  MITCOE  ['CATIA']   
1           1  1002     B.E.               Mechanical  MITCOE  ['CATIA']   
2           2  1003     B.E.               Mechanical  MITAOE  ['CATIA']   
3           3  1004     B.E.               Mechanical  MITCOE  ['CATIA']   
4           4  1005     B.E.               Mechanical  MITCOE  ['CATIA']   

  key_skills_str  
0          CATIA  
1          CATIA  
2          CATIA  
3          CATIA  
4          CATIA

In [3]:
user_dataset = pd.read_csv("data/userData.csv", encoding= 'unicode_escape')
user_dataset.head()

Unnamed: 0 userid degree_1        degree_1_specializations  \
0          0   1001     B.E.  Computer Science & Engineering   
1          1   1002     B.E.  Computer Science & Engineering   
2          2   1003     B.E.  Computer Science & Engineering   
3          3   1004     B.E.  Computer Science & Engineering   
4          4   1005     B.E.  Computer Science & Engineering   

                                 known_languages  \
0              ['English', ' Marathi', ' Hindi']   
1                              ['Hindi English']   
2                                    ['Missing']   
3  ['English', ' Hindi', ' Marathi', ' Marwari']   
4   ['English', 'Hindi  ', 'Kashmiri ', ' Urdu']   

                                          key_skills  \
0  ['C', ' Java', ' Keras', ' Flask', ' Deep Lear...   
1  ['Java', ' Neural Networks', ' AI', ' Python',...   
2                                        ['Missing']   
3  ['XML', ' C', ' Java', ' Data Structures', ' P...   
4  ['XML', ' Word', ' Data Structures', ' Communi...   

                                    career_objective  \
0  Computer Engineering student with good technic...   
1  Interested in working under company offering A...   
2                                            Missing   
3  Currently a final year student of Computer Eng...   
4  To have a growth oriented and challenging care...   

                                      key_skills_str  
0  C,  Java,  Keras,  Flask,  Deep Learning,  Sel...  
1  Java,  Neural Networks,  AI,  Python,  Html5, ...  
2                                            Missing  
3  XML,  C,  Java,  Data Structures,  Python,  Mo...  
4  XML,  Word,  Data Structures,  Communication, ...

In [4]:
ratings_df = pd.read_csv("data/ratingData.csv", encoding= 'unicode_escape')
ratings_df.head()

course_id  user_id  rating
0       2001     1001       5
1       2001     1002       3
2       2001     1003       1
3       2001     1004       0
4       2001     1005       2

## Collaborative Filtering

In [5]:
#Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will 
#like a particular product or service those users have used/experienced but I have not.
#I will not be implementing Collaborative Filtering from scratch. Instead, I will use the Surprise library 
#that used extremely powerful algorithms like Singular Value Decomposition (SVD) to minimise RMSE (Root Mean Square Error)
#and give great recommendations.
#Implementation of SVD for surprise library is given on this link

Lower values of RMSE indicate better fit. RMSE is a good measure of how accurately the model predicts the response. It's the most important criterion for fit if the main purpose of the model is prediction.
Based on a rule of thumb, it can be said that RMSE values between **0.2 and 0.5** shows that the model can relatively predict the data accurately.

There is no correct value for MSE. Simply put, the lower the value the better and 0 means the model is perfect.

MAE: A metric that tells us the **mean absolute difference** between the predicted values and the actual values in a dataset. The lower the MAE, the better a model fits a dataset.

### Using Surprise and testing with different algorithms (SVD, KnnBasic, KnnBaseline, KnnWithMeans, KnnWithZScore) to find the best one to use based on MAE and RMAE scores. 

In [6]:
#using surprise
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings_df[['user_id','course_id','rating']], reader)

#split into training and test sets using 80:20 ratio
raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)                

ratio = int(len(raw_ratings)*0.8)
train_raw = raw_ratings[:ratio] 
test_raw = raw_ratings[ratio:] 

data.raw_ratings = train_raw       
trainset = data.build_full_trainset() 
testset = data.construct_testset(test_raw)

#create dict for different models 
models=[SVD(), KNNBasic(), KNNBaseline(), KNNWithMeans(), KNNWithZScore()] 
results = {} #to store the scores

#perform cross validation of MAE and RMSE for all models
for model in models:
    #kfold set to 5
    crossval_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)  
    
    #saving and renaming appropraitely
    result = pd.DataFrame.from_dict(crossval_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE', 'fit_time': 'Fit Time', 'test_time': 'Test Time'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result
    
#printing all models results
all_models = pd.DataFrame.from_dict(results)
all_models.T.sort_values(by='RMSE') #models sorted by RMSE

MAE      RMSE  Fit Time  Test Time
matrix_factorization.SVD   1.115679  1.476805  0.027076   0.037816
knns.KNNBaseline           1.297740  1.652069  0.007825   0.056866
knns.KNNWithMeans          1.306738  1.667789  0.003587   0.025603
knns.KNNWithZScore         1.303735  1.669556  0.004053   0.020787
knns.KNNBasic              1.337374  1.693927  0.002181   0.053122

####  Using the SVD model to generate collaborative filtering recommendations since it has the least MAE and RMSE results

In [17]:
#Since the svd recommender is making recommmendations from the ratings dataset, 
#its recommemnding courses the users have already done
def svd_cf_recommendations(user):
    
    #usisng the "best" parameters found using the gridsearchcv method from experiments notebook 
    model = SVD(n_factors = 90, n_epochs = 20, lr_all = 0.005, reg_all = 0.2)
    model.fit(trainset)
    
    #building test set and predict the ratings
    testset = trainset.build_anti_testset()
    predictions = model.test(testset)
    
    #save into dataframe
    rating_predictions = pd.DataFrame(predictions)
    
    #find the top 30 predictions for given user
    user_predictions = rating_predictions[rating_predictions['uid'] == user].\
                         sort_values(by="est", ascending = False).head(30)
    
    #create recommendations array and append user predictions as recommendations
    recommendations = []
    recommendations.append(list(user_predictions['iid']))
    recommendations = recommendations[0]
    
    return(recommendations)

In [8]:
ratings_df[ratings_df['user_id']== 1001]

course_id  user_id  rating
0          2001     1001       5
20         2002     1001       3
40         2003     1001       4
60         2004     1001       2
80         2005     1001       0
...         ...      ...     ...
8380       2420     1001       0
8400       2421     1001       4
8420       2422     1001       0
8440       2423     1001       2
8460       2424     1001       0

[424 rows x 3 columns]

In [9]:
ratings_df[ratings_df['course_id']== 2150]

course_id  user_id  rating
2980       2150     1001       4
2981       2150     1002       5
2982       2150     1003       3
2983       2150     1004       2
2984       2150     1005       4
2985       2150     1006       5
2986       2150     1007       3
2987       2150     1008       4
2988       2150     1009       4
2989       2150     1010       5
2990       2150     1011       3
2991       2150     1012       5
2992       2150     1013       5
2993       2150     1014       2
2994       2150     1015       4
2995       2150     1016       3
2996       2150     1017       5
2997       2150     1018       5
2998       2150     1019       3
2999       2150     1020       3

In [18]:
svd_cf_recommendations(1001) #input is a user and output is a course!

#1001 is a user (shown above no course is 1001)
#2082 is course since no user rating is found (check above)

[2109,
 2397,
 2115,
 2099,
 2379,
 2153,
 2327,
 2218,
 2089,
 2315,
 2362,
 2206,
 2345,
 2401,
 2326,
 2272,
 2301,
 2086,
 2102,
 2025,
 2335,
 2384,
 2184,
 2375,
 2119,
 2373,
 2386,
 2364,
 2422,
 2387]

In [11]:
svd_cf_recommendations(2150) #svd doeesnt work with input of courses

[]

In [13]:
svd_cf_recommendations("2150") #svd doeesnt work with input of courses

[]

## Get course recommendations that the user has not rated

In [218]:
#using surprise
reader = Reader(rating_scale=(1, 5)) #ratings of 0 are considered unrated 
data = Dataset.load_from_df(ratings_df[['user_id','course_id','rating']], reader)

#split into training and test sets using 80:20 ratio
raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)                

ratio = int(len(raw_ratings)*0.8)
train_raw = raw_ratings[:ratio] 
test_raw = raw_ratings[ratio:] 

data.raw_ratings = train_raw       
trainset = data.build_full_trainset() 
testset = data.construct_testset(test_raw)

In [219]:
#Since the svd recommender is making recommmendations from the ratings dataset, 
#its recommemnding courses the users have already done
def svd_cf_recommendations(user):
    
    #usisng the "best" parameters found using the gridsearchcv method from experiments notebook 
    model = SVD(n_factors = 90, n_epochs = 20, lr_all = 0.005, reg_all = 0.2)
    model.fit(trainset)
    
    #building test set and predict the ratings
    testset = trainset.build_anti_testset()
    predictions = model.test(testset)
    
    #save into dataframe
    rating_predictions = pd.DataFrame(predictions)
    #print(rating_predictions)
    
    #find the predictions for the given user
    user_predictions = rating_predictions[rating_predictions['uid'] == user].\
                         sort_values(by="est", ascending = False)
        
    #removing courses already done by user (assuming they are rated 1 to 5)  
    finalcourserecs = [] #to store the final course recs
    
    #uid is the user and iid is the course id
    for i, row in user_predictions.iterrows():
        #get rating details
        rating_dets = ratings_df[(ratings_df['course_id'] == row[1]) & (ratings_df['user_id'] == user)]
        
        if rating_dets["rating"].values[0] == 0:
            #print("Not rated!")
            finalcourserecs.append(row[1])
    
    return(finalcourserecs)

In [220]:
svd_cf_recommendations(1001) #input is a user and output is a course!

[2088,
 2258,
 2268,
 2202,
 2133,
 2364,
 2400,
 2016,
 2393,
 2387,
 2357,
 2340,
 2032,
 2418,
 2176,
 2390,
 2245,
 2196,
 2389,
 2420,
 2226,
 2249,
 2307,
 2424,
 2302,
 2407,
 2350,
 2124]